# MNIST 手写数字识别与 MLP

本 Notebook 使用 TensorFlow/Keras 实现了一个多层感知机 (MLP)，包含三层隐藏层，用于分类 MNIST 手写数字数据集。

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子以确保可重复性
tf.random.set_seed(42)
np.random.seed(42)

## 1. 加载和预处理 MNIST 数据集

加载 MNIST 数据集并将像素值归一化到 [0, 1] 范围。

In [ ]:
# 加载 MNIST 数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 归一化像素值到 [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 打印数据集形状
print(f'训练数据形状: {x_train.shape}')
print(f'测试数据形状: {x_test.shape}')

## 2. 可视化样本数据

展示训练集中的几张样本图像。

In [ ]:
# 绘制前几张图像
plt.figure(figsize=(10, 2))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title(f'标签: {y_train[i]}')
    plt.axis('off')
plt.savefig('mnist_samples.png')

## 3. 构建 MLP 模型

创建一个包含三层隐藏层的 MLP：
- 输入层：将 28x28 图像展平为 784 维向量
- 隐藏层 1：256 个神经元，ReLU 激活函数
- 隐藏层 2：128 个神经元，ReLU 激活函数
- 隐藏层 3：64 个神经元，ReLU 激活函数
- 输出层：10 个神经元（对应每个数字），Softmax 激活函数

In [ ]:
# 构建 MLP 模型
model = Sequential([
    Flatten(input_shape=(28, 28)),  # 输入层
    Dense(256, activation='relu'),  # 第一隐藏层
    Dense(128, activation='relu'),  # 第二隐藏层
    Dense(64, activation='relu'),   # 第三隐藏层
    Dense(10, activation='softmax') # 输出层
])

# 编译模型
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 显示模型摘要
model.summary()

## 4. 训练模型

在训练数据上训练 MLP，设置 10 个 epoch，并使用 20% 的验证集分割。

In [ ]:
# 训练模型
history = model.fit(x_train, y_train, epochs=10, batch_size=128,
                    validation_split=0.2, verbose=1)

## 5. 评估模型

在测试集上评估模型，并绘制训练和验证的准确率及损失曲线。

In [ ]:
# 在测试数据上评估
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'测试准确率: {test_accuracy:.4f}')

# 绘制训练和验证准确率
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='训练准确率')
plt.plot(history.history['val_accuracy'], label='验证准确率')
plt.title('模型准确率')
plt.xlabel('Epoch')
plt.ylabel('准确率')
plt.legend()
plt.grid(True)

# 绘制训练和验证损失
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='训练损失')
plt.plot(history.history['val_loss'], label='验证损失')
plt.title('模型损失')
plt.xlabel('Epoch')
plt.ylabel('损失')
plt.legend()
plt.grid(True)

plt.savefig('training_metrics.png')

## 6. 进行预测

在几张测试图像上测试模型并显示预测结果。

In [ ]:
# 在测试数据上进行预测
predictions = model.predict(x_test[:5])
predicted_labels = np.argmax(predictions, axis=1)

# 显示预测结果
plt.figure(figsize=(10, 2))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(x_test[i], cmap='gray')
    plt.title(f'预测: {predicted_labels[i]}\n真实: {y_test[i]}')
    plt.axis('off')
plt.savefig('predictions.png')